### Tweet Score = Tweet sentiment * Tweet relevance
### Tweet sentiment score = (-1, 1)
### Tweet relevance = sum of the salience scores of the specified keywords
### Twitter data per day = average of all tweet scores OR sum of all tweet scores?

In [ ]:
import pandas as pd
from definitions import GOOGLE_CRED_PATH, DATA_DIR
from google.cloud import language_v1
from google.oauth2 import service_account

In [ ]:
credentials = service_account.Credentials.from_service_account_file(GOOGLE_CRED_PATH)
client = language_v1.LanguageServiceClient(credentials=credentials)
document_type = language_v1.Document.Type.PLAIN_TEXT
encoding_type = language_v1.EncodingType.UTF8

In [ ]:
twitter_data = pd.read_csv(DATA_DIR + '/elonmusk_new.csv')
tweet_df = pd.DataFrame({'Sentiment': [], 'Magnitude': [], 'Entity to Salience': []})

In [ ]:
for tweet in twitter_data.tweet:
    document = {'content': tweet, 'type_': document_type}
    request = {'document': document, 'encoding_type': encoding_type}
    entity_dict = {}

    try:
        sentiment_response = client.analyze_sentiment(request=request)
    except:
        sentiment = magnitude = float('nan')
    else:
        sentiment = sentiment_response.document_sentiment.score
        magnitude = sentiment_response.document_sentiment.magnitude

        entity_response = client.analyze_entities(request=request)
        for entity in entity_response.entities:
            entity_dict[entity.name] = entity.salience

    tweet_results = {'Sentiment': sentiment, 'Magnitude': magnitude, 'Entity to Salience': entity_dict}
    tweet_df = tweet_df.append(tweet_results, ignore_index=True)

In [ ]:
twitter_data[['Sentiment', 'Magnitude', 'Entity to Salience']] = tweet_df
twitter_data.to_csv(DATA_DIR + '/twitter.csv')